In [1]:
from bs4 import BeautifulSoup
import requests
import re
import datetime

In [2]:
def text_in_tag(tag):
    string = tag.get_text()
    string = string.rstrip()
    string = string.lstrip()
    return string

In [49]:
base_url = 'https://www.imdb.com/title/tt5420376/'     # Riverdale
base_url1 = 'https://www.imdb.com/title/tt1837492/'    # 13RW
base_url2 = 'https://www.imdb.com/title/tt3107288/'    # Flash
base_url3 = 'https://www.imdb.com/title/tt6077448/'    # Sacred Games
base_url4 = 'https://www.imdb.com/title/tt0903747/'    # Breaking Bad
base_url5 = 'https://www.imdb.com/title/tt1119644/'    # Fringe
base_url6 = 'https://www.imdb.com/title/tt0108778/'    # FRIENDS
base_url7 = 'https://www.imdb.com/title/tt0944947/'    # Game Of Thrones
response = requests.get(base_url7)

In [50]:
soup = BeautifulSoup(response.content, 'html.parser')
#print(soup.prettify())

In [51]:
div = soup.find_all('div', id='titleStoryLine')[0]
temp = div.find('div', class_='inline canwrap')
temp = temp.find('span')
story = text_in_tag(temp)
story

'In the mythical continent of Westeros, several powerful families fight for control of the Seven Kingdoms. As conflict erupts in the kingdoms of men, an ancient enemy rises once again to threaten them all. Meanwhile, the last heirs of a recently usurped dynasty plot to take back their homeland from across the Narrow Sea.'

In [52]:
temp2 = div.find('div', class_='see-more inline canwrap')
temp2 = temp2.find_all('a')
genre = []
for x in temp2:
    g = text_in_tag(x)
    if g.lower().startswith('see all'):
        break
    genre.append(g)
print("Genre -> ")
print(genre)

Genre -> 
['dragon', 'based on novel', 'bloody violence', 'twins incest', 'politics']


In [53]:
div = soup.find_all('div', class_='seasons-and-year-nav')[0]
a_list = div.find_all('a')
latest_season = a_list[0].get_text()
latest_season_link = a_list[0]['href']
print("Latest season -> ",latest_season)
print("Link -> ", latest_season_link)

Latest season ->  8
Link ->  /title/tt0944947/episodes?season=8&ref_=tt_eps_sn_8


In [54]:
episodes_url = 'https://www.imdb.com' + latest_season_link
response2 = requests.get(episodes_url)
soup2 = BeautifulSoup(response2.content, 'html.parser')
#print(soup2.prettify())

In [55]:
div = soup2.find_all('div', class_='airdate')

#Today's date
current_date = datetime.datetime.now().date()
print("Today's date = ", current_date)
c_date = datetime.datetime.strftime(current_date, "%d %B %Y")
print("Today's date = ", c_date)
print()

#Indexing months
short = ['Jan.', 'Feb.', 'Mar.', 'Apr.', 'Jun.', 'Jul.', 'Aug.', 'Sep.', 'Oct.', 'Nov.', 'Dec.']
long = ['January', 'February', 'March', 'April', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
release_dates = {}

for i,x in enumerate(div):
    #Episode number
    print('e{}'.format(i+1))
    date_string = text_in_tag(x)
    
    #Convert short form of months to full expansion, as string
    for y in short:
        if y in date_string:
            date_string = date_string.replace(y, long[short.index(y)])
    print('Release Date -> ', date_string)
    
    date = ''
    month = ''
    year = ''
    if date_string[0].isalpha():
        #Convert to datetime format, get MONTHS and YEARS,
        date_string = datetime.datetime.strptime(date_string, "%B %Y")
        month = date_string.month
        year = date_string.year
        if(month == current_date.month) and (year == current_date.year):
            print('The next season airs this month -> ', month, year)
            break
        elif(month > current_date.month) and (year >= current_date.year):
            print('The next season airs this month -> ', month, year)
            break
        
    elif date_string[3].isalpha():
        date_string = datetime.datetime.strptime(date_string, "%d %B %Y")
        date = date_string.day
        month = date_string.month
        year = date_string.year
        if(date == current_date.day) and (month == current_date.month) and (year == current_date.year):
            print("The episode airs today!! -> ", date, month, year)
            break
        elif (date > current_date.day) and (month >= current_date.month) and (year >= current_date.year):
            print("The next episode airs on -> ", date, month, year)
            break
        
    else:
        date_string = datetime.datetime.strptime(date_string, "%Y")
        year = date_string.year
        if year == current_date.year:
            print("The next season begins this year -> ", year)
            break
        if year > current_date.year:
            print("The next season begins in -> ", year)
            break
    print()
if year < current_date.year:
    print("The show has finished streaming all its episodes!")

Today's date =  2018-10-11
Today's date =  11 October 2018

e1
Release Date ->  2019
The next season begins in ->  2019
